In [1]:
import os
import psycopg2
import pandas as pd
import json

## Note:

1. The **URL** to the google drive containing the **dec-jan_flights.csv** file: https://drive.google.com/drive/folders/1m61S5WMcSpdXV6pq3k7srXOQIr0fnlOv?usp=sharing
    - The code to create this file is below.


In [2]:
def connect_db():
    '''
    Creating connection to DB
    '''
    try:
        conn = psycopg2.connect(
                host=os.environ['MID_PROJ_DB_HOST'],
                database=os.environ['MID_PROJ_DB'],
                user=os.environ['MID_PROJ_DB_USERNAME'],
                password=os.environ['MID_PROJ_DB_PASSWORD'],
                port='5432')
        print('Db Connected succsefuly')
        return conn
    except (Exception, Error) as error:
        print("Error while connecting")

In [3]:
def query_db(query,conn):
    cursor = conn.cursor()
    cursor.execute(query)
    return {'data': cursor.fetchall(), 'columns': [desc[0].strip() for desc in cursor.description]} 

In [4]:
def write_to_file(csv_name,query_output):
    df = pd.DataFrame(query_output['data'])
    df.columns = query_output['columns']
    df.to_csv(csv_name + '.csv', sep=",", index = False)

In [5]:
conn = connect_db()

Db Connected succsefuly


In [6]:
# Note: this is one method of data sampling, we may want to explore others
# This method primarily focuses on looking at the 2 week time period just before when we want to predict, over a yearly basis

flights_jan_18_query="select * from flights where fl_date > '2017-12-31' and fl_date < '2018-01-08' LIMIT 250000;"
flights_dec_18_query="select * from flights where fl_date > '2018-12-26' and fl_date < '2019-01-01' LIMIT 250000;"
flights_jan_19_query="select * from flights where fl_date > '2018-12-31' and fl_date < '2019-01-08' LIMIT 250000;"
flights_dec_19_query="select * from flights where fl_date > '2019-12-24' and fl_date < '2020-01-01' LIMIT 250000;"

### Looking specifically at the TWO weeks before the predicted time period, over the previous two years

In [ ]:
# FOUR different files

flights_jan_18 = query_db(flights_jan_18_query, conn)
write_to_file('flights_jan_18',flights_jan_18)

flights_dec_18 = query_db(flights_dec_18_query, conn)
write_to_file('flights_dec_18',flights_dec_18)

flights_jan_19 = query_db(flights_jan_19_query, conn)
write_to_file('flights_jan_19',flights_jan_19)

flights_dec_19 = query_db(flights_dec_19_query, conn)
write_to_file('flights_dec_19',flights_dec_19)

In [ ]:
flights_jan18 = pd.read_csv('flights_jan_18.csv', sep = ',')
flights_dec18 = pd.read_csv('flights_dec_18.csv', sep = ',')
flights_jan19 = pd.read_csv('flights_jan_19.csv', sep = ',')
flights_dec19 = pd.read_csv('flights_dec_19.csv', sep = ',')

In [ ]:
flights = [flights_jan18, flights_dec18, flights_jan19, flights_dec19]
flights = pd.concat(flights)

In [ ]:
flights = flights.reset_index()

In [ ]:
flights.head()

In [ ]:
flights.to_csv('dec-jan_flights.csv')

In [ ]:
# flights_jan_dec_19_18_query="select * from flights where fl_date > '2017-12-31' and fl_date < '2018-01-08' union select * from flights  where fl_date > '2018-11-30' and fl_date < '2019-02-01' union select * from flights where  fl_date > '2019-11-30' and fl_date < '2020-01-01' LIMIT 300000;"
# flights_jan_dec_19_18 = query_db(flights_jan_dec_19_18_query,conn)

In [ ]:
# write_to_file('flights_jan_dec_19_18',flights_jan_dec_19_18)

In [ ]:
# passengers_jan_dec_19_18_query = "select * from passengers where \"month\" in (1, 12) and year in (2018, 2019) LIMIT 300000;"
# passengers_jan_dec_19_18 = query_db(passengers_jan_dec_19_18_query,conn)

In [ ]:
# write_to_file('passengers_jan_dec_19_18',passengers_jan_dec_19_18)

In [ ]:
# flights = pd.read_csv('flights_jan_dec_19_18.csv', sep = ',')

In [ ]:
# flights